<a href="https://colab.research.google.com/github/bachvu98/Policy-NLP/blob/master/Model_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Basic imports
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import *
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
url = "/content/drive/My Drive/OPP-115/OPP-115/annotations/"
files = [f for f in listdir(url) if f.endswith('.csv')]

In [16]:
names = """annotation_id 
        batch_id 
        annotator_id 
        policy_id 
        segment_id 
        category_name 
        attributes_value_pairs 
        date 
        policy_url""".split()

types = {'annotation_id': np.int,
        'batch_id': str,
        'annotator_id': np.int,
        'policy_id': np.int,
        'segment_id': np.int,
        'category_name': str}

ann_list = []
pids = []
for f in files:
    df = pd.read_csv(url + f,header=None,names=names,na_values={'date': 'Not specified'},
                 parse_dates=[7],index_col=0)
    ann_list.append(df)
    pids.append(np.int(f.split('_')[0]))

In [23]:
ann_list

[                                                     batch_id  ...                                         policy_url
 annotation_id                                                  ...                                                   
 17980          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
 17982          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
 17983          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
 17984          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
 17985          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
 ...                                                       ...  ...                                                ...
 17306          test_category_labeling_highlight

In [24]:
annotations = pd.concat(ann_list,axis=0,keys=pids,names=['Policy UID','annotation_id'])
annotations = annotations.drop('policy_id',axis=1)

In [26]:
annotations.head(20)

batch_id  ...                                         policy_url
Policy UID annotation_id                                                  ...                                                   
1666       17980          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17982          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17983          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17984          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17985          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17986          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17987          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17988          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17989          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17990          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17991          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17992          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17993          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17994          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17995          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17996          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17997          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17998          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           17999          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...
           18000          test_category_labeling_highlight_fordham_aaaaa  ...  http://www.wsmv.com/story/18990/this-web-sites...

[20 rows x 7 columns]